In [15]:
%load_ext autoreload
%autoreload 2

import itertools
import os
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import pickle
from tqdm import tqdm
from scipy import stats
from multiprocessing import Pool

from graph_generation import GenerateGraph
from models import GCN, view_parameters, new_parameters, modify_parameters, Inits
from train import Trainer, Dataset
from embedding import Embedding, NodeEmbedding
import experiments
from experiments import (init_GCN, init, init_model_dataset, experiment_node_embedding, 
                         elaborate_embeddings, elaborate_embeddings_2, elaborate_embedding_pergraph, separate_embedding_by_classes,
                         run_with_weights, init_with_weights, make_video, take_corr, plot_epoch, get_corrs_training)

from config_valid import Config, TrainingMode
from GridConfigurations import GridConfigurations

import torch
from torch_geometric.loader import DataLoader
import tensorflow as tf

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.height', 500)

## Variabili per la grid search

In [18]:
config_file = "configurations/classification_cm.yml"
config_class = Config(config_file)

diz_trials = {#'model.GCNneurons_per_layer' : [ [1, 16, 1], 
              #                               [1, 6, 1],
              #                               [1, 2, 1]],
              #'graph_dataset.Num_grafi_per_tipo': [200, 100, 50] ,
              'model.neurons_last_linear': [[2,1], [2, 2, 2, 2, 1], [20, 20, 20, 20, 1]],
              'training.learning_rate': [0.01, 0.005, 0.001]
}
#diz_trials = {'graph_dataset.Num_grafi_per_tipo': [200] ,
#              'model.neurons_last_linear': [[5,1], [3,3,1]]}

gc = GridConfigurations(config_class, diz_trials)
gc.make_configs()
len(gc.configs)

9

In [20]:
gc.config_dataframe.head(10)

device graph_dataset                                              \
     NaN       ERmodel regular confmodel continuous_p       list_p   
0    gpu         False   False      True         True  [0.05, 0.3]   
1    gpu         False   False      True         True  [0.05, 0.3]   
2    gpu         False   False      True         True  [0.05, 0.3]   
3    gpu         False   False      True         True  [0.05, 0.3]   
4    gpu         False   False      True         True  [0.05, 0.3]   
5    gpu         False   False      True         True  [0.05, 0.3]   
6    gpu         False   False      True         True  [0.05, 0.3]   
7    gpu         False   False      True         True  [0.05, 0.3]   
8    gpu         False   False      True         True  [0.05, 0.3]   

                                                                            \
       range_p   Num_nodes Num_grafi_per_tipo Num_grafi_totali list_degree   
0  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   
1  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   
2  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   
3  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   
4  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   
5  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   
6  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   
7  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   
8  [0.05, 0.6]  [500, 500]                100              400     [4, 50]   

                                  training                                     \
  random_node_feat list_exponents     mode epochs batch_size percentage_train   
0            False   [-2.5, -4.5]    mode2   1000        100              0.7   
1            False   [-2.5, -4.5]    mode2   1000        100              0.7   
2            False   [-2.5, -4.5]    mode2   1000        100              0.7   
3            False   [-2.5, -4.5]    mode2   1000        100              0.7   
4            False   [-2.5, -4.5]    mode2   1000        100              0.7   
5            False   [-2.5, -4.5]    mode2   1000        100              0.7   
6            False   [-2.5, -4.5]    mode2   1000        100              0.7   
7            False   [-2.5, -4.5]    mode2   1000        100              0.7   
8            False   [-2.5, -4.5]    mode2   1000        100              0.7   

                                                                    \
  earlystop_patience epochs_model_checkpoint every_epoch_embedding   
0                 50                    [-1]                  True   
1                 50                    [-1]                  True   
2                 50                    [-1]                  True   
3                 50                    [-1]                  True   
4                 50                    [-1]                  True   
5                 50                    [-1]                  True   
6                 50                    [-1]                  True   
7                 50                    [-1]                  True   
8                 50                    [-1]                  True   

             model                                                            \
  last_layer_dense GCNneurons_per_layer  neurons_last_linear freezeGCNlayers   
0             True   [1, 32, 32, 16, 1]               [2, 1]           False   
1             True   [1, 32, 32, 16, 1]               [2, 1]           False   
2             True   [1, 32, 32, 16, 1]               [2, 1]           False   
3             True   [1, 32, 32, 16, 1]      [2, 2, 2, 2, 1]           False   
4             True   [1, 32, 32, 16, 1]      [2, 2, 2, 2, 1]           False   
5             True   [1, 32, 32, 16, 1]      [2, 2, 2, 2, 1]           False   
6             True   [1, 32, 32, 16, 1]  [20, 20, 20, 20, 1]         

def start_from_saved_dataset_parameters():
    with open('init_parameters.pickle', 'rb') as file:
        previous_par = pickle.load(file)
    with open('dataset.pickle', 'rb') as file:
        dataset = pickle.load(file)

    config_file = "configurations/classification_cm.yml"
    config_c = Config(config_file)
    #config_c.conf['model']['freezeGCNlayers'] = True

    model = init_GCN(config_c, verbose=False)
    modify_parameters(model, previous_par)

    trainer = Trainer(model, config_c)
    trainer.load_dataset(dataset, percentage_train=config_c.conf['training']['percentage_train'], shuffle=False, parallel=False)
    return trainer, config_c

trainer, config_c = start_from_saved_dataset_parameters()

In [21]:
for i, c in enumerate(gc.configs):
    gc.config_dataframe.loc[i, ('risultati','test_loss')] = '_'
    gc.config_dataframe.loc[i, ('risultati','test_accuracy')] = '_'
    gc.config_dataframe.loc[i, ('risultati','correlation_class0')] = '_'
    gc.config_dataframe.loc[i, ('risultati','correlation_class1')] = '_'

In [22]:


#outfile = "df_configs_new.csv"

#with open('init_parameters.pickle', 'rb') as file:
#    previous_par = pickle.load(file)
#with open('dataset.pickle', 'rb') as file:
#    dataset = pickle.load(file)

# prendo un nuovo set di parametri per questa nuova architettura
model = init_GCN(config_class, verbose=False)
new_par = new_parameters(model, method=Inits.xavier_uniform)

# nuovo dataset più grosso da mantenere per i training
gg = GenerateGraph(config_class)
dataset = gg.initialize_dataset(parallel=True)
trainer = Trainer(model, config_class)
trainer.load_dataset(dataset, percentage_train=config_class.conf['training']['percentage_train'], shuffle=False, parallel=False)

        
for i, c in enumerate(gc.configs):
    print(f'Run {i+1}/{len(gc.configs)}')
    try:       
        init_with_weights(c, new_par, trainer)
        trainer.launch_training()
        
        # serve per get_corrs_training
        experiments.t = trainer
        experiments.c = c
        c_e_0, c_e_1 = get_corrs_training()
    
        gc.config_dataframe.at[i, ('risultati','test_loss')] = trainer.test_loss_list
        gc.config_dataframe.at[i, ('risultati','test_accuracy')] = trainer.accuracy_list
        gc.config_dataframe.at[i, ('risultati','correlation_class0')] = c_e_0[0]
        gc.config_dataframe.at[i, ('risultati','correlation_class1')] = c_e_1[0]
        
        #gc.config_dataframe.iloc[i:i+1].to_csv(outfile, index=False, mode='a', header=not os.path.exists(outfile))

    except Exception as e: 
        print(repr(e))
        raise(e)


Generating dataset...
Dataset generated


100%|████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 47.54it/s]


Tempo impiegato: 4.22522497177124
Run 1/9
Run 2/9
Run 3/9
Run 4/9
Run 5/9
Run 6/9
Run 7/9
Run 8/9
Run 9/9


In [15]:
gc.config_dataframe

device graph_dataset                                              \
     NaN       ERmodel regular confmodel continuous_p       list_p   
0    gpu         False   False      True         True  [0.05, 0.3]   
1    gpu         False   False      True         True  [0.05, 0.3]   
2    gpu         False   False      True         True  [0.05, 0.3]   

                                                                              \
       range_p     Num_nodes Num_grafi_per_tipo Num_grafi_totali list_degree   
0  [0.05, 0.6]  [1000, 1000]                600              400     [4, 50]   
1  [0.05, 0.6]  [1000, 1000]                600              400     [4, 50]   
2  [0.05, 0.6]  [1000, 1000]                600              400     [4, 50]   

                                  training                                  \
  random_node_feat list_exponents     mode learning_rate epochs batch_size   
0            False   [-2.5, -4.5]    mode2          0.01   5000         50   
1            False   [-2.5, -4.5]    mode2          0.01   5000         50   
2            False   [-2.5, -4.5]    mode2          0.01   5000         50   

                                                               \
  percentage_train earlystop_patience epochs_model_checkpoint   
0              0.7                 50                    [-1]   
1              0.7                 50                    [-1]   
2              0.7                 50                    [-1]   

                                   model                       \
  every_epoch_embedding last_layer_dense GCNneurons_per_layer   
0                  True            False   [1, 32, 32, 16, 1]   
1                  True            False   [1, 32, 32, 16, 1]   
2                  True            False   [1, 32, 32, 16, 1]   

                                                                              \
   neurons_last_linear freezeGCNlayers autoencoder put_batchnorm put_dropout   
0               [2, 1]           False       False         False       False   
1      [2, 2, 2, 2, 1]           False       False         False       False   
2  [20, 20, 20, 20, 1]           False       False         False       False   

                                                   logging  \
  final_pool_aggregator node_features_dim train_step_print   
0                  True                 1              100   
1                  True                 1              100   
2                  True                 1              100   

                                           risultati  \
                                           test_loss   
0  [0.01893960220946206, 0.020342233777046203, 0....   
1  [0.018939602043893604, 0.020342233611477745, 0...   
2  [0.018939602043893604, 0.02034223328034083, 0....   

                                                      \
                                       test_accuracy   
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                      \
                                  correlation_class0   
0  [-0.9301530441987759, -0.9300804346641053, -0....   
1  [-0.9301530441987759, -0.9300804346641053, -0....   
2  [-0.9301530441987759, -0.9300804346641053, -0....   

                                                      
                                  correlation_class1  
0  [-0.9674967180464958, -0.9673429536231619, -0....  
1  [-0.9674967180464958, -0.9673429536231619, -0....  
2  [-0.9674967180464958, -0.9673429536231619, -0....

In [23]:
outfile = "df_configs_new7.csv"
gc.config_dataframe.to_csv(outfile)#, index=False, mode='a', header=not os.path.exists(outfile))

In [ ]:
# alla fine per concatenare dataframe con run diversi
df_tot = pd.concat([yaml_df, yaml_df2])